In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install scipy

In [7]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

def predict_ratings(ratings_matrix, k=50):
    mean_user_ratings = np.nanmean(ratings_matrix, axis=1, keepdims=True)
    ratings_matrix_normalized = ratings_matrix - mean_user_ratings
    ratings_matrix_normalized = np.nan_to_num(ratings_matrix_normalized)  # Handle NaNs properly

    # Perform SVD
    U, sigma, Vt = svds(ratings_matrix_normalized, k=min(k, min(ratings_matrix.shape) - 1))
    sigma = np.diag(sigma)
    
    # Reconstruct predicted ratings
    predicted_ratings = np.dot(np.dot(U, sigma), Vt) + mean_user_ratings
    return predicted_ratings

def recommend_movies(user_id, ratings_df, predicted_ratings, top_n=5):
    user_index = user_id - 1  # Assuming user IDs start from 1
    user_predictions = predicted_ratings[user_index]
    
    # Get movies user hasn't rated yet
    rated_movies = ratings_df[ratings_df['userId'] == user_id]['movieId'].tolist()
    recommendations = [(i + 1, user_predictions[i]) for i in range(len(user_predictions)) if (i + 1) not in rated_movies]
    
    # Sort by predicted rating
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    return pd.DataFrame(recommendations[:top_n], columns=['movieId', 'predicted_rating'])

# Example Usage
data = {
    'userId': [1,1,1,2,2,3,3,4,4,5],
    'movieId': [1,2,3,1,3,2,4,3,5,1],
    'rating': [4,5,3,5,4,2,3,5,4,3]
}
ratings_df = pd.DataFrame(data)

# Create user-item matrix
num_users = ratings_df['userId'].max()
num_movies = ratings_df['movieId'].max()
ratings_matrix = np.full((num_users, num_movies), np.nan)

for row in ratings_df.itertuples():
    ratings_matrix[row.userId - 1, row.movieId - 1] = row.rating

# Predict ratings
predicted_ratings = predict_ratings(ratings_matrix)

# Recommend movies for user 1
recommendations = recommend_movies(4, ratings_df, predicted_ratings)
print(recommendations)


   movieId  predicted_rating
0        2               4.5
1        4               4.5
2        1               4.5
